In [43]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import urllib.request
import re
import cv2
import matplotlib.pyplot as plt
import os 

In [10]:
#  get list of recipe
# https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/3435
# select range
def get_list_recipe(start, end):
    data = []
    links = []
    for i in range(start, end):
        url = requests.get("https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/" + str(i))
        soup = BeautifulSoup(url.content, 'lxml')
        # select the title
        for title in soup.find_all("h3"):
            data.append(title.get_text())
            links.append(f'https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/{i}')
    return {"nama_makanan" : data, "links" : links}
# functon to concat 
def concat_data(data1:dict, data2:dict):
    data_concat = pd.concat([pd.DataFrame(data1), pd.DataFrame(data2)])
    return data_concat 
# function to convert
def convert_excel(data1:dict, data2:dict): 
    data = concat_data(data1, data2)
    data = pd.DataFrame(data).to_excel("list_recipe.xlsx", index = False)
#  function to read data
def read_data():
    data = pd.read_excel("list_recipe.xlsx")
    return data

In [11]:
# get list of recipe

# all data index
# data1 = get_list_recipe(start = 3436, end = 3467)
# data2 = get_list_recipe(start = 59670, end = 59996)


data1 = get_list_recipe(start = 3436, end = 3440)
data2 = get_list_recipe(start = 59670, end = 59680)
convert_excel(data1, data2)


In [15]:
food_list = read_data()
food_list.head(10)

,nama_makanan,links
0,Spicy Eggplant (Terong Balado),https://cosylab.iiitd.edu.in/recipedb/search_r...
1,How to Make Beef Satay,https://cosylab.iiitd.edu.in/recipedb/search_r...
2,Indonesian Pork Satay,https://cosylab.iiitd.edu.in/recipedb/search_r...
3,Spicy Sambal Shrimp,https://cosylab.iiitd.edu.in/recipedb/search_r...
4,Nutmeg Tea Cookies (Kue Kering),https://cosylab.iiitd.edu.in/recipedb/search_r...
5,INDONESIAN FOOD RECIPES : SOTO KUDUS,https://cosylab.iiitd.edu.in/recipedb/search_r...
6,Kaya (Coconut Egg Custard),https://cosylab.iiitd.edu.in/recipedb/search_r...
7,Nasi Kuning,https://cosylab.iiitd.edu.in/recipedb/search_r...
8,Beef Rendang,https://cosylab.iiitd.edu.in/recipedb/search_r...
9,Telo ( Surinam-Indonesian),https://cosylab.iiitd.edu.in/recipedb/search_r...


In [16]:
str1 =  "nasi goreng"
str1 = str1.replace(" ",  "+")
str1

'nasi+goreng'

In [40]:
food_list.shape

(14, 2)

In [47]:
# get image 
def download_image(food_data: pd.DataFrame):
    #  looping for nama makanan 
    for food in food_data['nama_makanan']: 
        url = requests.get(f'https://www.bing.com/images/search?q={food}&form=HDRSC3')
        soup = BeautifulSoup(url.content, 'lxml')

        get_image = soup.find_all(name = "a", attrs={"class" : "itm"})
        src2_value = []
        # print(url.status_code)
        if get_image: 
            # print(get_image)
            for i in get_image: 
                img_tag = i.find('img')
                if img_tag:
                # get the src value 
                    src2_value.append(img_tag.get('src2'))
            # start downloading
            for i in range(len(src2_value)):
                # check  path
                if os.path.exists(f'./{food}'):
                    full_path = f'./{food}/{food}{i + 1}.jpg'
                    urllib.request.urlretrieve(src2_value[i], full_path)
                else:
                    # if path is not exist
                    os.mkdir(f"./{food}")
                    full_path = f'./{food}/{food}{i + 1}.jpg'
                    urllib.request.urlretrieve(src2_value[i], full_path)

    #  not detecting image
    else: 
        print("there is no img detected")
    print("succesfully get " + str(len(src2_value)) + " images")



In [48]:
# url_image =
download_image(food_data=food_list)

NotADirectoryError: [WinError 267] The directory name is invalid: './INDONESIAN FOOD RECIPES : SOTO KUDUS'